In [26]:
!pip install botorch

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import time
import torch
import numpy as np
import botorch
from botorch import fit_gpytorch_model
from botorch.models import SingleTaskGP, HeteroskedasticSingleTaskGP
from gpytorch.mlls.sum_marginal_log_likelihood import ExactMarginalLogLikelihood

torch.set_default_dtype(torch.float64)

In [28]:
lower_bounds = [-3, -3]
upper_bounds = [3, 3]
bounds = [lower_bounds, upper_bounds]
bounds = torch.tensor(bounds, dtype=torch.float)


def blackbox_func(x):
    res = x[0]**3 + x[1]**3
    var = torch.rand(1).item()/5 # var in [0.5, 1.5]
    res_noise = np.random.normal(loc=res, scale=np.sqrt(var))
    
    return res_noise, var

blackbox_func([2.0, 2.0])

(16.390802848518486, 0.16833492580389176)

In [29]:
def generate_initial_data(n=10):
    # generate training data
    train_x = torch.rand(n, 2)
    train_y = []
    train_y_var = []

    for i in range(n):
        t_y, t_y_var = blackbox_func(train_x[i])
        train_y.append(t_y)
        train_y_var.append(t_y_var)

    train_y = torch.tensor(train_y).reshape(-1,1)
    train_y_var = torch.tensor(train_y_var).reshape(-1,1)


    return train_x, train_y, train_y_var
    
    
def initialize_model(train_x, train_y, train_y_var, state_dict=None):
    # define models for objective and constraint
    model = SingleTaskGP(train_x, train_y)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)

    # load state dict if it is passed
    if state_dict is not None:
        model.load_state_dict(state_dict)
        
    return mll, model

In [30]:
from botorch import fit_gpytorch_model
from botorch.optim import optimize_acqf
from botorch.acquisition.monte_carlo import qNoisyExpectedImprovement
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling import IIDNormalSampler, SobolQMCNormalSampler

def get_next_points(train_x, train_y, train_y_var, best_y, bounds, n_points):    
    train_x_mean = torch.mean(train_x, dim=0)
    train_x_std = torch.std(train_x, dim=0)
    train_x = (train_x-train_x_mean)/train_x_std


    train_y_mean = torch.mean(train_y, dim=0)
    train_y_std = torch.std(train_y, dim=0)
    train_y = (train_y-train_y_mean)/train_y_std

    
    mll, model = initialize_model(train_x, train_y, train_y_var)

    fit_gpytorch_model(mll, retain_graph=True)

    sampler = SobolQMCNormalSampler(2048)
    qNEI = qNoisyExpectedImprovement(model, train_x, sampler)

    candidates = optimize_acqf(
        acq_function=qNEI,
        bounds=bounds,
        q=n_points,
        num_restarts=20,
        raw_samples=500
    )

    return candidates

In [33]:
from botorch.exceptions import InputDataWarning
import warnings
warnings.filterwarnings("ignore", category=InputDataWarning)

NUM_ITERATIONS = 300

train_x, train_y, train_y_var = generate_initial_data(n=3)
best_observed = torch.max(train_y)

for iteration in range(1, NUM_ITERATIONS + 1):    
        print("="*30)
        print("Iteration number = ", iteration)
        t0 = time.time()

        
        candidate = get_next_points(train_x, train_y, train_y_var, 0, bounds, 1)
        candidate_x = candidate[0]
        candidate_y, candidate_y_var = blackbox_func(candidate_x[0])

        train_x = torch.cat([train_x, candidate_x])
        train_y = torch.cat([train_y, torch.tensor(candidate_y).reshape(-1,1)])
        train_y_var = torch.cat([train_y_var, torch.tensor(candidate_y_var).reshape(-1,1)])
        best_observed = torch.max(train_y)

        i = torch.argmax(train_y)
        print(f"{train_x[i]=}")
        print(f"{train_y[i]=}")
        if train_y[i] > 26.5:
                break
        t = time.time()
        print(f"Got in {t-t0} seconds")

SyntaxError: f-string: empty expression not allowed (1425114923.py, line 27)

In [32]:

n=10
lower_bounds = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
upper_bounds = [10, 10, 10, 2, 2, 2]
bounds = [lower_bounds, upper_bounds]
bounds = torch.tensor(bounds)
